In [70]:
import numpy as np

In [71]:
# Number of desired iterations
N=10**4

# only about 1/6 of random conditions satisfy reasonability requirements
N=6*N

# All values have min 0, adjust values below to change upper bound
nu=10
nd=10
A=100
beta=1 #needs to be  <=1
delta=1 #needs to be <=1
Iu=100
Id=100
I0=100

# generate 6Nx8 matrix of random seed evaluation points
randos=np.random.rand(N,8)
# used to normalize values
scalars=np.diag([nu,nd,A,beta,delta,Iu,Id,I0])

#do the normalization
inputs=np.matmul(randos,scalars)
print(inputs.shape)

# Drop points with impossible conditions, nu<nd,Iu<Id,Iu<I0
inputs = inputs[inputs[:,0]>inputs[:,1]]
inputs = inputs[inputs[:,5]>inputs[:,6]]
inputs = inputs[inputs[:,5]>inputs[:,7]]
print(inputs.shape)

(60000, 8)
(9984, 8)


In [72]:
# Create utility at each point

def compute_U(input_row):
  nu,nd,A,beta,delta,Iu,Id,I0=input_row
  Q=(1-delta)*A+I0 #constant to make formulas shorter
  # Compute utilities
  uA=(Q+I0)*nd+beta*((1-delta)*(Q+I0)+2*Id)*nd
  uB=(Q)*nu+beta*((1-delta)*(Q)+2*Iu)*nd
  uC=(Q+I0)*nd+beta*((1-delta)*(Q+I0)+Id)*nu
  uD=(Q)*nu+beta*((1-delta)*Q+Iu)*nu

  #return each utility
  return [uA, uB, uC, uD]

#compute utility for each strategy at every input
U=np.apply_along_axis(compute_U,1,inputs) 
U.shape

(9984, 4)

In [73]:
def rank(LETTERS): #function to define a potential ranking of outcomes
  # accepts a vector [n1,n2,n3,n4] giving which is preferred to which
  TruthTable=(U[:,LETTERS[0]]>U[:,LETTERS[1]])& \
  (U[:,LETTERS[1]]>U[:,LETTERS[2]])& \
  (U[:,LETTERS[2]]>U[:,LETTERS[3]])
  #returns a Nx1 vector of whether or not the proposed ranking is true for the set of inputs
  return TruthTable

import itertools
#makes all permutations, which is all possible rankings
LETTERS=np.array(list(itertools.permutations([0,1,2,3]))) 

In [74]:
# for each possible ranking compute when that's possible
OUT=np.apply_along_axis(rank,1,LETTERS)

In [75]:
# see which rankings occur a non-zero amount of times
#STRATS=np.array(list(itertools.permutations(["low,low>","high,low>","low,high>","high,high>"]))) 
STRATS=np.array(list(itertools.permutations(["A","B","C","D"]))) 
Outcome=np.append(STRATS,np.apply_along_axis(np.count_nonzero,1,OUT).reshape(-1,1),axis=1)
print(Outcome)

[['A' 'B' 'C' 'D' '400']
 ['A' 'B' 'D' 'C' '0']
 ['A' 'C' 'B' 'D' '1042']
 ['A' 'C' 'D' 'B' '0']
 ['A' 'D' 'B' 'C' '0']
 ['A' 'D' 'C' 'B' '0']
 ['B' 'A' 'C' 'D' '380']
 ['B' 'A' 'D' 'C' '334']
 ['B' 'C' 'A' 'D' '67']
 ['B' 'C' 'D' 'A' '50']
 ['B' 'D' 'A' 'C' '715']
 ['B' 'D' 'C' 'A' '702']
 ['C' 'A' 'B' 'D' '289']
 ['C' 'A' 'D' 'B' '78']
 ['C' 'B' 'A' 'D' '40']
 ['C' 'B' 'D' 'A' '22']
 ['C' 'D' 'A' 'B' '29']
 ['C' 'D' 'B' 'A' '131']
 ['D' 'A' 'B' 'C' '0']
 ['D' 'A' 'C' 'B' '0']
 ['D' 'B' 'A' 'C' '0']
 ['D' 'B' 'C' 'A' '4528']
 ['D' 'C' 'A' 'B' '0']
 ['D' 'C' 'B' 'A' '1177']]


In [76]:
# add input data
out_data=np.append(inputs,OUT.T,axis=1)
A_win=np.sum(OUT.T[:,0:5],axis=1)
B_win=np.sum(OUT.T[:,6:11],axis=1)
C_win=np.sum(OUT.T[:,12:17],axis=1)
D_win=np.sum(OUT.T[:,18:23],axis=1)


In [78]:
import pandas as pd 
from google.colab import files

names=np.array(list(itertools.permutations(["A","B","C","D"])))
names=np.apply_along_axis(lambda d: d[0] + d[1] + d[2]+d[3] , 1, names)
colnames=np.append(['nu','nd','iA','beta','delta','Iu','Id','I0'],names)

out_data=pd.DataFrame(out_data,columns=colnames)
out_data['A_win']=A_win
out_data['B_win']=B_win
out_data['C_win']=C_win
out_data['D_win']=D_win

def func(row):
    if row['A_win'] == 1:
        return "A"
    elif row['B_win'] ==1:
        return "B"
    elif row['C_win'] ==1:
        return "C"
    else:
        return 'D'

out_data['winner'] = out_data.apply(func, axis=1)
out_data.to_csv("TryData.csv")
files.download("TryData.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
out_data

,nu,nd,iA,beta,delta,Iu,Id,I0,ABCD,ABDC,ACBD,ACDB,ADBC,ADCB,BACD,BADC,BCAD,BCDA,BDAC,BDCA,CABD,CADB,CBAD,CBDA,CDAB,CDBA,DABC,DACB,DBAC,DBCA,DCAB,DCBA,A_win,B_win,C_win,D_win,winner
0,3.443336,1.914566,19.215997,0.614012,0.317950,83.517192,27.079493,41.678406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
1,4.622638,0.543789,46.731366,0.736443,0.438399,96.973041,26.770823,93.830172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
2,7.977625,6.879699,56.363551,0.521917,0.277907,98.263557,31.419547,4.664256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,B
3,8.829830,2.710730,78.946480,0.181800,0.816717,94.975293,29.283739,84.610664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
4,7.673235,1.859363,37.163390,0.713460,0.554819,84.473621,13.010279,8.650459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9979,2.939761,2.359705,49.868606,0.973202,0.399723,88.023059,15.257451,59.690836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,D
9980,5.481347,1.744679,48.586503,0.911994,0.992314,92.538914,29.764597,40.803359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
9981,5.145169,0.627386,93.496451,0.627119,0.345578,77.730853,28.500333,2.152699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
9982,7.326029,3.861958,75.642208,0.008510,0.036104,84.543752,48.037121,37.986341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,D
